# Saved scripts is assumed to have scripts with the following stuff in it

### breakfast_weekday_1 : Prepare coffee and toast , Eat coffee and toast , Put b1 utensils in sink , Wash b1 utensils
### breakfast_weekday_2 : Prepare cereal and milk , Eat cereal and milk , Put b2 utensils in sink , Wash b2 utensils
### breakfast_weekend : Prepare peanut butter sandwich , Eat sandwich on sofa , Put bw utensils in sink , Wash bw utensils
### icecream : Serve icecream , Eat icecream , Put away icecream , Put ic utensils in sink , Wash ic utensils
### laundry : Collect clothes , Wash clothes , Put away clothes
### meal : Prepare cheese sandwich , Eat cheese sandwich , Return ml utensils , Wash ml utensils
### meal_long : Chop vegetables , Take out chicken , Cook chicken and vegetables , Serve meal , Eat meal , Return lml utensils , Wash lml utensils
### read : Get a book , Read the book , Return the book
### sleep : Get alarm clock and book , Read and sleep

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import glob
import json
import sys
sys.path.append('simulation')

In [ ]:
from evolving_graph.scripts import Script, parse_script_line
import evolving_graph.utils as utils
from evolving_graph.scripts import read_script, read_script_from_string, read_script_from_list_string, ScriptParseException, Action
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph

dt = 10 #min

In [ ]:
dataset_dir = 'saved_datasets/my_scripts'
graph_file = '{}/initial_common.json'.format(dataset_dir)


action_paths = glob.glob('{}/executable_programs/initial_common/*'.format(dataset_dir))
print('Actions found : ',[os.path.basename(a) for a in action_paths])

action_scripts = {}
actions_vocab = {}
relevant_object_ids = set()

## Add rooms as relevant ids
with open (graph_file,'r') as f:
    graph_dict = json.load(f)
for n in graph_dict['nodes']:
    if n['category'] == 'Rooms':
        relevant_object_ids.add(n['id'])

for action_path in action_paths:
    file_name = os.path.join(action_path,'complete.txt')
    action = os.path.basename(action_path)
    action_scripts[action] = []
    actions_vocab[action] = []
    with open(file_name) as f:
        lines = []
        index = 1
        for line in f:
            if line.startswith('##'):
                actions_vocab[action].append(line[2:].strip())
                action_scripts[action].append(lines)
                lines = []
                index = 1
            if '[' not in line:
                continue
            line = line.strip()
            if len(line) > 0 and not line.startswith('#'):
                scr_line = parse_script_line(line, index, custom_patt_params = r'\<(.+?)\>\s*\(.+?\.(.+?)\)')
                if scr_line.object():
                    relevant_object_ids.add(scr_line.object().instance)
                if scr_line.subject():
                    relevant_object_ids.add(scr_line.subject().instance)
                ## Ignore sit and standup actions for now
                if scr_line.action in [Action.SIT, Action.STANDUP]:
                    continue
                lines.append(scr_line)
                index += 1
        action_scripts[action].append(lines)
        action_scripts[action] = action_scripts[action][1:]
        
for action in actions_vocab:
    print('~~~',action,'~~~')
    for name, script in zip(actions_vocab[action], action_scripts[action]):
        print(name)
        for l in script:
            print('-',str(l))

print(f'Found {len(relevant_object_ids)} relevant objects')


## Helper class to specify time options for an action

In [ ]:
import random

class ActionTime:
    def __init__ (self, mean_t, std_t, min_t=None, max_t=None):
        self.mean_t = mean_t
        self.std_t = std_t
        self.min_t = (mean_t-3*std_t) if min_t is None else min_t 
        self.max_t = (mean_t+3*std_t) if max_t is None else max_t 
        assert self.std_t>0, 'Std time should be positive'
        assert self.min_t<self.max_t, 'Minimum time should be less than maximum'

    def sample (self, n=1):
        t = set()
        while len(t) < n:
            val = int(round(np.random.normal(self.mean_t, self.std_t)))
            if val > self.min_t and val < self.max_t:
                t.add(val)
        t=list(t)
        t.sort()
        return t

    def definite (self, n=1):
        return list(np.linspace(self.mean_t-self.std_t, self.mean_t+self.std_t, n))


## Helper functions for time related calculations

In [ ]:
def time_internal(mins, hrs, days=0, weeks=0):
    return int(round(((((weeks*7+days)*24)+hrs)*60+mins)/dt))

def time_external(in_t):
    in_t = in_t*dt
    mins = in_t % 60
    in_t = in_t // 60
    hrs = in_t % 24
    in_t = in_t // 24
    days = in_t % 7
    in_t = in_t // 7
    weeks = in_t
    return(weeks, days, hrs, mins)

## Schedules and constraints

In [ ]:
t=time_internal

weekday_times = []
weekday_times.append([
    {'prob':0.5, 'action':'breakfast_weekday_1', 'time': ActionTime(mean_t=t(0,8), std_t=t(30,0), min_t=t(0,6), max_t=t(0,9))},
    {'prob':0.5, 'action':'breakfast_weekday_2', 'time': ActionTime(mean_t=t(0,8), std_t=t(30,0), min_t=t(0,6), max_t=t(0,9))}
])
weekday_times.append([
    {'prob':0.8, 'action':'meal', 'time': ActionTime(mean_t=t(0,19), std_t=t(0,1), min_t=t(0,18), max_t=t(0,20))},
    {'prob':0.2, 'action':'meal_long', 'time': ActionTime(mean_t=t(0,19), std_t=t(0,1), min_t=t(0,18), max_t=t(0,20))}
])
weekday_times.append([
    {'prob':1.0, 'action':'sleep', 'time': ActionTime(mean_t=t(0,22), std_t=t(30,0), min_t=t(0,21), max_t=t(0,23))}
])


weekend_times = []
weekend_times.append([
    {'prob':0.8, 'action':'breakfast_weekend', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))},
    {'prob':0.1, 'action':'breakfast_weekday_1', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))},
    {'prob':0.1, 'action':'breakfast_weekday_2', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))}
])
weekend_times.append([
    {'prob':0.8, 'action':'read', 'time': ActionTime(mean_t=t(0,12), std_t=t(0,3), min_t=t(0,9), max_t=t(0,16))},
    {'prob':0.2, 'action':'laundry', 'time': ActionTime(mean_t=t(0,12), std_t=t(0,3), min_t=t(0,9), max_t=t(0,16))}
])
weekend_times.append([
    {'prob':0.8, 'action':'meal_long', 'time': ActionTime(mean_t=t(30,13), std_t=t(0,1), min_t=t(0,11), max_t=t(0,15))},
    {'prob':0.2, 'action':'meal', 'time': ActionTime(mean_t=t(30,13), std_t=t(0,1), min_t=t(0,11), max_t=t(0,15))}
])
weekend_times.append([
    {'prob':0.8, 'action':'read', 'time': ActionTime(mean_t=t(0,18), std_t=t(0,3), min_t=t(0,15), max_t=t(0,21))},
    {'prob':0.2, 'action':'laundry', 'time': ActionTime(mean_t=t(0,18), std_t=t(0,3), min_t=t(0,15), max_t=t(0,21))}
])
weekend_times.append([
    {'prob':0.8, 'action':'meal_long', 'time': ActionTime(mean_t=t(30,19), std_t=t(0,1), min_t=t(0,17), max_t=t(0,21))},
    {'prob':0.2, 'action':'meal', 'time': ActionTime(mean_t=t(30,19), std_t=t(0,1), min_t=t(0,17), max_t=t(0,21))}
])
weekend_times.append([
    {'prob':0.7, 'action':'icecream', 'time': ActionTime(mean_t=t(0,20), std_t=t(0,2), min_t=t(0,18), max_t=t(0,22))},
    # 0.3 : None
])
weekend_times.append([
    {'prob':1.0, 'action':'sleep', 'time': ActionTime(mean_t=t(0,23), std_t=t(0,1), min_t=t(0,21), max_t=t(0,24))}
])


constraints = []  #constraint function, error message
constraints.append((lambda sch: all([a[2]<b[2] for a in sch for b in sch if a[0]=='meal' and a[1]==1 and b[0]=='icecream' and b[1]==1]),  'Must eat meal before eating ice cream'))
constraints.append((lambda sch: all([a[2]<b[2] for a in sch for b in sch if a[0]=='meal_long' and a[1]==4 and b[0]=='icecream' and b[1]==1]),  'Must eat long meal before eating ice cream'))
constraints.append((lambda sch: all([a[2]<=b[2] for a in sch for b in sch if b[0]=='sleep' and b[1]==1]),  'Must do everything before sleeping'))
constraints.append((lambda sch: len(set([t[2] for t in sch])) == len([t[2] for t in sch]),  'No two actions can have the same timestamp'))


## Sampling helper functions

In [ ]:
def sample_action_type(probs):
    r = random.random()
    for i,p in enumerate(probs):
        r -= p
        if r<0:
            return i
    return None

def definite_action(_):
    return 0

def get_one_action_with_times(action_options, stochastic=True):
    get_action = sample_action_type if stochastic else definite_action
    action_idx = get_action([a['prob'] for a in action_options])
    if action_idx is None:
        return []
    action = action_options[action_idx]['action']
    action_times = action_options[action_idx]['time'].sample(len(actions_vocab[action]))
    return ([(action, i, t) for i,t in enumerate(action_times)])

## Test

In [ ]:
def get_random_schedule(weekend=False):
    template = weekend_times if weekend else weekday_times
    sch = []
    while True:
        for seg in template:
            sch = sch + get_one_action_with_times(seg, stochastic=True)
        good_sch = all([c[0](sch) for c in constraints])
        if good_sch:
            sch.sort(key=lambda x: x[2])
            return sch
        sch = []

sch = get_random_schedule(weekend=True)
for s in (sch):
    print (s[0], s[1],s[2], time_external(s[2]))

for c in constraints:
    if not c[0](sch):
        print(c[1])


## Graph helpers

In [ ]:
def clip_graph(graph):
    clipped = {'nodes':[], 'edges':[]}
    use_ids = relevant_object_ids
    clipped['nodes'] = [n for n in graph['nodes'] if n['id'] in relevant_object_ids]
    clipped['edges'] = [e for e in graph['edges'] if e['from_id'] in relevant_object_ids and e['to_id'] in relevant_object_ids]
    return clipped


def class_from_id(graph, id):
    lis = [n['class_name'] for n in graph['nodes'] if n['id']==id]
    if len(lis) > 0:
        return lis[0]
    else:
        return 'None'

def get_graph_difference(g1,g2):
    diff = {'nodes':{},'edges':{}}

    def character_edge(e):
        return class_from_id(g1, e['from_id']) == 'character' or class_from_id(g1, e['to_id']) == 'character'

    diff['edges']['removed'] = [e for e in g1['edges'] if e not in g2['edges'] and not character_edge(e)]
    diff['edges']['added'] = [e for e in g2['edges'] if e not in g1['edges'] and not character_edge(e)]
    diff['nodes']['removed'] = [n for n in g1['nodes'] if n['id'] not in [n2['id'] for n2 in g2['nodes']]]
    diff['nodes']['added'] = [n for n in g2['nodes'] if n['id'] not in [n2['id'] for n2 in g1['nodes']]]

    return diff

def update_changed_ids(g1,g2):
    diff = get_graph_difference(g1,g2)
    ids = [n['id'] for n in diff['nodes']['removed']+diff['nodes']['added']]
    ids = ids + [e['from_id'] for e in diff['edges']['removed']+diff['edges']['added']]
    ids = ids + [e['to_id'] for e in diff['edges']['removed']+diff['edges']['added']]
    for id in ids:
        relevant_object_ids.add(id)

## Helper class to realize a schedule

```
path_input = 'dataset/my_scripts/withoutconds/meal_long/complete.txt'
translated_path = 'dataset/my_scripts/initial_common.json'
%run -i 'simulation/evolving_graph/check_programs.py'

```

In [ ]:
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph
import evolving_graph.utils as utils


class Routine():
    def __init__(self, sch = None, init_graph_file = graph_file):
        self.schedule = sch
        with open (init_graph_file,'r') as f:
            self.init_graph_dict = json.load(f)
        self.init_graph = EnvironmentGraph(self.init_graph_dict)
        self.script = []
        self.timestamps = [0]
        self.relevant_states = [True]
        for seg in self.schedule:
            self.script += action_scripts[seg[0]][seg[1]]
            self.timestamps.append(seg[2])
            self.relevant_states += [False for _ in range(len(self.script) - len(self.relevant_states))] 
            self.relevant_states[-1] = True
        self.state = None


    def execute(self):
        name_equivalence = utils.load_name_equivalence()
        executor = ScriptExecutor(self.init_graph, name_equivalence)
        success, _, state_list = executor.execute(Script(self.script), w_graph_list=True)
        for s1,s2 in zip(state_list[:-1], state_list[1:]):
            update_changed_ids(s1,s2)
        if success:
            graphs = [state for admit,state in zip(self.relevant_states,state_list) if admit]
            return self.timestamps, graphs
        else:
            return None, None

In [ ]:
import time
random.seed(time.time())
routine = Routine(get_random_schedule(weekend=True))
times,graphs = routine.execute()
graphs_clipped = [clip_graph(graph) for graph in graphs]

In [ ]:
print(len(graphs_clipped))
print(len(times))
a_graph = graphs_clipped[0]
print(len(a_graph['nodes']))
all_classes = [n['class_name'] for n in a_graph['nodes']]
classes = set(all_classes)
for c in classes:
    print(c,all_classes.count(c))
print(len(relevant_object_ids))

In [ ]:
def print_graph_difference(g1,g2):
    
    diff = get_graph_difference(g1,g2)
    print(diff['nodes'])
    print('Edges removed')
    for e in diff['edges']['removed']:
        if e['relation_type'] != 'CLOSE':
            print (class_from_id(g1,e['from_id']),e['relation_type'],class_from_id(g1,e['to_id']))
    print('Edges added')
    for e in diff['edges']['added']:
        if e['relation_type'] != 'CLOSE':
            print (class_from_id(g1,e['from_id']),e['relation_type'],class_from_id(g1,e['to_id']))
    

for g1, g2 in zip(graphs[:-1], graphs[1:]):
    print()
    print()
    print_graph_difference(g1,g2)